In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b

In [2]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [3]:
market = Market()
speculation_db = ADatabase("btc_spec")

In [4]:
prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["ticker"] = "BTC"

In [5]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [6]:
simulation["week"] = simulation["week"] + 1

In [7]:
simulation.dropna()

,year,week,ticker,adjclose,y,0,1,2,3,4,...,12,13,xgb_prediction,xgb_score,cat_prediction,cat_score,tf_prediction,tf_score,prediction,training_years
0,2019,2,BTC,4736.226196,3810.276681,4736.226196,3859.500139,3861.717494,3351.422468,3662.343959,...,6426.877232,6576.962891,5655.628906,0.960422,4662.508112,0.979311,4412.187988,0.936085,4910.108336,4
1,2019,3,BTC,3810.276681,3665.388532,3810.276681,4736.226196,3859.500139,3861.717494,3351.422468,...,6521.534319,6426.877232,4746.499512,0.960422,4606.479174,0.979311,3795.431641,0.936085,4382.803442,4
2,2019,4,BTC,3665.388532,3593.255755,3665.388532,3810.276681,4736.226196,3859.500139,3861.717494,...,6482.364328,6521.534319,4281.716309,0.960422,4523.436108,0.979311,3712.627441,0.936085,4172.593286,4
3,2019,5,BTC,3593.255755,3476.508754,3593.255755,3665.388532,3810.276681,4736.226196,3859.500139,...,6355.445592,6482.364328,4266.304688,0.960422,5001.927465,0.979311,3277.658691,0.936085,4181.963615,4
4,2019,6,BTC,3476.508754,3538.131871,3476.508754,3593.255755,3665.388532,3810.276681,4736.226196,...,6438.662947,6355.445592,3988.576904,0.960422,4820.264432,0.979311,3051.636230,0.936085,3953.492522,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2022,49,BTC,16846.483817,17073.143136,16846.483817,16374.710798,16651.560826,17542.640485,20672.421038,...,20300.163505,20008.611328,18580.201172,0.966133,32455.979399,0.967298,16771.654297,0.940144,22602.611623,4
206,2022,50,BTC,17073.143136,17195.546317,17073.143136,16846.483817,16374.710798,16651.560826,17542.640485,...,20275.589007,20300.163505,18580.201172,0.966133,34340.744298,0.967298,16007.896484,0.940144,22976.280652,4
207,2022,51,BTC,17195.546317,16782.936663,17195.546317,17073.143136,16846.483817,16374.710798,16651.560826,...,19061.802455,20275.589007,18580.201172,0.966133,32354.968625,0.967298,17041.339844,0.940144,22658.836547,4
208,2022,52,BTC,16782.936663,24376.750733,16782.936663,17195.546317,17073.143136,16846.483817,16374.710798,...,19302.997489,19061.802455,17917.869141,0.966133,33305.301807,0.967298,17652.615234,0.940144,22958.595394,4


In [8]:
simulation = prices.merge(simulation.drop("adjclose",axis=1),on=["year","week","ticker"],how="left")

In [9]:
sim = simulation.dropna()

In [10]:
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]

In [11]:
sim.head(100)

,date,open,high,low,close,adjclose,volume,year,quarter,week,...,13,xgb_prediction,xgb_score,cat_prediction,cat_score,tf_prediction,tf_score,prediction,training_years,delta
1573,2019-01-07,4078.584961,4092.613525,4020.894043,4025.248291,4025.248291,5228625637,2019,1,2,...,6576.962891,5655.628906,0.960422,4662.508112,0.979311,4412.187988,0.936085,4910.108336,4.0,0.219827
1574,2019-01-08,4028.472168,4109.020996,3996.955322,4030.847900,4030.847900,5306593305,2019,1,2,...,6576.962891,5655.628906,0.960422,4662.508112,0.979311,4412.187988,0.936085,4910.108336,4.0,0.218133
1575,2019-01-09,4031.552002,4068.403564,4022.662842,4035.296387,4035.296387,5115905225,2019,1,2,...,6576.962891,5655.628906,0.960422,4662.508112,0.979311,4412.187988,0.936085,4910.108336,4.0,0.216790
1576,2019-01-10,4034.411377,4064.066650,3659.174561,3678.924561,3678.924561,6874143796,2019,1,2,...,6576.962891,5655.628906,0.960422,4662.508112,0.979311,4412.187988,0.936085,4910.108336,4.0,0.334659
1577,2019-01-11,3674.015381,3713.881836,3653.069824,3687.365479,3687.365479,5538712865,2019,1,2,...,6576.962891,5655.628906,0.960422,4662.508112,0.979311,4412.187988,0.936085,4910.108336,4.0,0.331603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,2019-04-12,5061.200684,5103.274414,4955.852539,5089.539063,5089.539063,13675206312,2019,2,15,...,4736.226196,5809.061035,0.960422,6445.849117,0.979311,4436.134277,0.936085,5563.681476,4.0,0.093160
1669,2019-04-13,5088.850098,5127.122070,5061.589355,5096.586426,5096.586426,10823289598,2019,2,15,...,4736.226196,5809.061035,0.960422,6445.849117,0.979311,4436.134277,0.936085,5563.681476,4.0,0.091649
1670,2019-04-14,5095.758789,5184.016113,5053.568359,5167.722168,5167.722168,10391952498,2019,2,15,...,4736.226196,5809.061035,0.960422,6445.849117,0.979311,4436.134277,0.936085,5563.681476,4.0,0.076622
1671,2019-04-15,5167.321777,5196.606934,5024.069336,5067.108398,5067.108398,12290155061,2019,2,16,...,3810.276681,5809.061035,0.960422,6202.619516,0.979311,5210.041016,0.936085,5740.573855,4.0,0.132909


In [12]:
trades = []
reqs = [0.05,0.1,0.15]
signals = [0.05,0.1,0.15]
values = [True,False]
parameters = []
positions = 1
training_year = 4
for value in values:
    for signal in signals:
        for req in reqs:
            parameter = {"value":value,"signal":signal,"req":req,"training_years":training_year}
            parameters.append(parameter)

In [ ]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
for parameter in tqdm(parameters):
    current_sim = sim[sim["training_years"]==parameter["training_years"]].reset_index()
    iteration_trades = b.speculation_backtest(sim,parameter,start_date,end_date)
    speculation_db.store("trades",pd.DataFrame(iteration_trades))
speculation_db.disconnect()

  0%|                                                                                                                                                             | 0/18 [00:00<?, ?it/s]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

...

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

 72%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                         | 13/18 [11:20<04:54, 58.85s/it]

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi